# OpenSearch 설정하기

프로젝트를 위한 Amazon OpenSearch Service 설정 방법을 안내해드리도록 하겠습니다.

## OpenSearch CloudFormation 스택 배포하기

In [ ]:
import boto3
import yaml
import time

# CloudFormation 템플릿 불러오기
with open("manifest/setup-opensearch.yaml", 'r') as file:
    template = file.read()

# CloudFormation 클라이언트 초기화
cf_client = boto3.client('cloudformation')

# 스택 배포
stack_name = 'contextual-retrieval-opensearch'
try:
    response = cf_client.create_stack(
        StackName=stack_name,
        TemplateBody=template,
        Capabilities=['CAPABILITY_IAM']
    )
    print(f"스택 생성이 시작되었습니다. 스택 ID: {response['StackId']}")
except Exception as e:
    print(f"스택 생성 중 오류가 발생했습니다: {str(e)}")

## 스택 생성 대기하기

In [ ]:
def wait_for_stack(stack_name):
    while True:
        response = cf_client.describe_stacks(StackName=stack_name)
        status = response['Stacks'][0]['StackStatus']
        print(f"현재 상태: {status}")
        
        if status.endswith('COMPLETE'):
            break
        elif status.endswith('FAILED'):
            raise Exception("스택 생성에 실패했습니다")
            
        time.sleep(30)
    return response['Stacks'][0]

stack = wait_for_stack(stack_name)
outputs = {output['OutputKey']: output['OutputValue'] for output in stack['Outputs']}
print("\n스택 출력값:")
for key, value in outputs.items():
    print(f"{key}: {value}")

## 환경변수 업데이트하기

OpenSearch 도메인 엔드포인트와 인증정보로 .env 파일을 업데이트합니다

In [ ]:
import os
from dotenv import load_dotenv

# 도메인 엔드포인트에서 https:// 제거
domain_endpoint = outputs['DomainEndpoint'].replace('https://', '')

env_content = f"""
OPENSEARCH_USER={outputs['UserID']}
OPENSEARCH_PASSWORD={outputs['UserPassword']}
OPENSEARCH_DOMAIN={domain_endpoint}
"""

with open('.env', 'w') as f:
    f.write(env_content.strip())

print(".env 파일이 업데이트되었습니다")

## OpenSearch 연결 테스트

In [ ]:
from libs.common_utils import OpenSearch_Manager

# 환경변수 다시 불러오기
load_dotenv(override=True)

# 연결 테스트
os_manager = OpenSearch_Manager()
try:
    indices = os_manager._get_indices()
    print("OpenSearch에 성공적으로 연결되었습니다")
    print(f"사용 가능한 인덱스: {indices}")
except Exception as e:
    print(f"OpenSearch 연결 중 오류가 발생했습니다: {str(e)}")